## Seera

In [2]:
import numpy as np

import pandas as pd
import matplotlib.pyplot as plt

#algoritmos ia
from sklearn.neighbors import KNeighborsRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm 
from sklearn.tree import DecisionTreeRegressor

from sklearn import model_selection as ms
from sklearn.datasets import make_regression

In [81]:
#Carregando dados

data = pd.read_csv('datasets/seera.csv', skiprows=1,sep = ';')
data.head()

,ProjID,Year of project,Organization id,Organization type,Role in organization,Size of organization,Size of IT department,Customer organization type,Estimated duration,Actual duration,...,Requirement accuracy level,Technical documentation,Comments within the code,User manual,Required reusability,Performance requirements,Product complexity,Security requirements,Reliability requirements,Specified H/W
0,1,2015,1,1,1,16,7,13,2,3,...,?,1,3,1,2,1,2,2,3,1
1,2,2016,25,5,1,2,1,2,2,3,...,2,2,2,1,1,1,1,4,2,1
2,3,2008,2,5,3,2,2,3,3,5,...,3,2,2,2,4,4,3,5,2,3
3,4,2009,42,4,2,3,2,4,6,6,...,1,1,3,4,4,2,4,3,4,2
4,5,2016,42,4,2,3,2,4,12,24,...,1,1,3,4,4,4,3,3,4,2


## Preparando Dados

In [82]:
def one_hot_encoder(X_transform, transform_data): #X_transform = data, transform_data = colunas
    
    for col in transform_data:
        if col in X_transform.columns:
            cols = X_transform[col].unique() #retorna os valores da categoria que serão transformados em colunas 
            col_position = X_transform.columns.get_loc(col) #retorna posição da coluna, a partir dessa posição será adicionado as demais
            col_values = X_transform.pop(col) #retorna os valores da coluna primordial e apaga e remove 
            for c in cols: 
                X_transform.insert(col_position, str(col) + "_" + str(c).replace('b\'','').replace('\'',''), col_values.eq(c).astype('int'))
                col_position += 1

    return X_transform  
   

In [87]:
#Definindo os atributos que serão usados. +
cols_drop = [
'ProjID', 
'Year of project',
'Organization id',
'Organization type',
'Role in organization',
'Size of organization',
'Size of IT department',
'Customer organization type',
'Estimated  duration',
'Actual duration',
'% project gain (loss)',
'Development type',
'Application domain',
#Object points; +
'Other sizing method',
'Estimated size',
'Estimated effort',
#Actual effort; + 
'Contract maturity',
'Government policy impact',
'Economic instability impact',
'Organization management structure clarity',
'Developer hiring policy',
'Developer incentives policy ',
'Developer training',
#Development team management; +
'Top management support',
'Top management opinion of previous system',
'Clarity of manual system',
'User resistance',
'User computer experience', 
' Users stability ', 
' Requirment stability ', 
' Requirements flexibility ',
#Project manager experience;+
'Consultant availability',
'DBMS  expert availability',
#Precedentedness;+
'Software tool experience',
#Programmers experience in programming language;+
'Programmers capability ',
'Analysts capability ',
'Team selection',
#Team size; +
'Dedicated team members',
'Daily working hours',
'Team contracts',
'Team continuity ',
'Team cohesion',
'Income satisfaction',
'Schedule quality',
'Development environment adequacy',
'Tool availability ',
'Methodology',
'# Multiple programing languages ',
#Programming language used; +
'DBMS used',
'Technical stability',
'Open source software',
'Level of outsourcing',
'Outsourcing impact',
'Degree of software reuse ',
'Degree of risk management',
#Use of standards; +
'Degree of standards usage', 
' Process reengineering ',
#Requirement accuracy level;+
'Technical documentation',
'Comments within the code',
'User manual',
#Required reusability; +
#Performance requirements; +
#Product complexity; +
#Security requirements; +
#Reliability requirements; +
'Specified H/W'    
]

df = data.drop(cols_drop, axis=1)

,Object points,Actual effort,Development team management,Project manager experience,Precedentedness,Programmers experience in programming language,Team size,Programming language used,Use of standards,Requirement accuracy level,Required reusability,Performance requirements,Product complexity,Security requirements,Reliability requirements
1,19,1584,7,1,2,2,3,5,3,2,1,1,1,4,2
2,64,5280,6,2,4,1,6,3,3,3,4,4,3,5,2
3,246,5280,7,1,4,1,6,4,2,1,4,2,4,3,4
4,420,38016,2,1,4,1,9,24,3,1,4,4,3,3,4
5,177,9856,2,1,4,1,7,4,3,1,4,4,3,3,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,180,16016,8,2,3,3,9,6,3,3,1,2,2,2,1
115,65,792,2,1,1,3,2,1,3,4,4,2,1,2,3
117,11,880,2,1,2,2,2,7,3,1,1,1,1,2,1
118,67,3168,2,1,2,5,2,1,3,1,2,2,1,2,3


In [93]:
#Convertendo atributos categóricos em colunas. "OneHotEncoder"

transform_data = ['Programming language used']
df =  one_hot_encoder(df, transform_data)

#removendo linhas com atributos nulos, vazios ou caracteres

df = df.replace(r'[^0-9a-zA-Z:]+', np.nan, regex=True)
df = df.dropna()

#convertendo dados to_numeric, para evitar incompatibilidade

for c in range(df.shape[1]):
    col = df.columns[c]
    df[col] = pd.to_numeric(df[col]) 


"""
for c in range(df.shape[1]):
    try:
        col = df.columns[c]
        df[col] = pd.to_numeric(df[col]) 
    except ValueError:
        mask = pd.to_numeric(df[col], errors='coerce').isna() 
        #if possible missing values
        #mask = pd.to_numeric(df[col].fillna('0'), errors='coerce').isna() 
        L = df.loc[mask, col].tolist()
        #Not converted values are: Wrong date format
        print ("Not converted values are: " + ", ".join(L))

"""


#normalizando dados - #min-max df=(df-df.min())/(df.max()-df.min()) - #zscore df=(df-df.mean())/df.std()

df = (df-df.mean())/df.std()

#separando atributo classe ACT_Effort para entrada nos modelos

X = df.drop('Actual effort', axis=1)
y = df.take([1], axis=1)

,Actual effort
1,-0.559878
2,-0.264599
3,-0.264599
4,2.350724
5,0.100983
...,...
114,0.593114
115,-0.623152
117,-0.616121
118,-0.433330


## Treinando e avaliando o desempenho dos modelos

In [94]:
models = []
#Instanciando os modelos
models.append(['KNN', KNeighborsRegressor(n_neighbors=5)]) 
models.append(['MLP', MLPRegressor(activation='logistic', solver='sgd', alpha=0.02, max_iter=300, hidden_layer_sizes=200)])
models.append(['RFR', RandomForestRegressor()])
models.append(['SVR', svm.SVR()])

In [95]:
for model in models:
    cv = ms.KFold(n_splits=3, shuffle=True, random_state=1)
    scores = ms.cross_val_score(model[1], X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    #converter para positivo.
    scores = np.absolute(scores)
    s_mean = np.mean(scores)
    s_std = np.std(scores)
    
    print('-', model[0],'------------------------------------')
    print('Scores:', scores, 'MAE: %.4f' % (s_mean), 'STD: %.4f' % (s_std)) 

- KNN ------------------------------------
Scores: [0.59523588 0.4896843  0.35828635] MAE: 0.4811 STD: 0.0969
- MLP ------------------------------------
Scores: [0.53300645 0.54544126 0.36017971] MAE: 0.4795 STD: 0.0846
- RFR ------------------------------------
Scores: [0.37038441 0.39501849 0.31942451] MAE: 0.3616 STD: 0.0315
- SVR ------------------------------------
Scores: [0.57345056 0.53428572 0.33908886] MAE: 0.4823 STD: 0.1025
